In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from ast import literal_eval
from math import log, e


# Cargar Data

In [ ]:
path='../SOURCES/' 
path_out='%sCorpus.csv' %(path)
data = pd.read_csv(path_out, sep=",", header=0, converters={"Corpus": literal_eval})
data.head()

In [ ]:
clientes = data["Clientes"].values

In [ ]:
corpus = list(data["Corpus"])

# Sequitur para inferir reglas gramaticales

Aplicamos el algoritmo de sequitur para inferir las reglas gramaticales dado por los codigos de los comportamientos, que nos generara "reglas", definida por los simbolos que se repiten en la secuencia.


### Funciones de Apoyo

#### Otras Funciones

In [ ]:
# Verifica que los elementos de una lista esten contenidos en otra lista
def elementos_en_lista(lista1, lista2):
    for i in range(len(lista1)):
        if not(lista1[i] in lista2):
            return False    
    return True

#### Algoritmo de Sequitur

In [ ]:
from SequiturAlgorithm import run_sequitur

#### Descompresion de Sequitur

In [ ]:
# Retorma la logintud de cada regla extendida y la regla estendida
def descomprimir_sequitur(semilla_l,nom_reglas,rs_comprimida,rs_tamaño,rs_extendida):
    num_reglas = len(nom_reglas)
    simbolos_semilla_les = [str(element) for element in list(np.unique(semilla_l))]
    simbolos_derivados = []
    
    incompleto = True
    contador = 0
    
    # 1 prueba: valores on solo 2 elementos
    for i in range(num_reglas):
        if elementos_en_lista(rs_comprimida[i],simbolos_semilla_les):
            rs_extendida[i] = rs_comprimida[i]
            rs_tamaño[i] =  len(rs_extendida[i]) # = 2
            simbolos_derivados.append(nom_reglas[i])  # cargamos la lista de reglas extendidas ya reconstruidas

    while(incompleto):
        # 2 pruebas: iterativas de 2 mas elementos
        for i in range(num_reglas):
            # si su profundidad aun no esta completada
            if rs_tamaño[i]==0:
                # si sus e estan entre los simbolos ya registrados
                if elementos_en_lista(rs_comprimida[i],(simbolos_semilla_les+simbolos_derivados)):
                    # para cada elemento de la lista
                    for e in range(len(rs_comprimida[i])):
                        if rs_comprimida[i][e] in simbolos_semilla_les:
                            rs_extendida[i].append(rs_comprimida[i][e])
                        if rs_comprimida[i][e] in simbolos_derivados:
                            indice = nom_reglas.index(rs_comprimida[i][e])
                            rs_extendida[i] =  rs_extendida[i] + rs_extendida[indice]
                    rs_tamaño[i] =  len(rs_extendida[i])  # actualizamos valor de prifundidad
                    simbolos_derivados.append(nom_reglas[i])  # cargamos la lista de reglas extendida
        
        # 3 Comprobamos que todas las reglas esten llenas
        if not(0 in rs_tamaño):
            incompleto = False

    return rs_tamaño,rs_extendida

### Funcion Principal

In [ ]:
def generador_de_reglas(lista_original):
    # Ejecutamos Sequitur
    resp_sequitur = run_sequitur(lista_original)
    reglas = resp_sequitur[0]  # Extraemos solo la lista de reglas (Ver Alg. Sequitur)
    incidencia = resp_sequitur[1]
    
    # Creamos listas para cada regla: nombres,exp comprimidad, numero_incidencias, longitud, exp extendida
    r_nombre = []
    r_comprimida = []
    r_incidencias = []
    r_longitud = []
    r_extendida = []

    for i in range(len(reglas)):
        nombre = 'R'+str(i) 
        profundidad = 0
        # print(nombre,' -> ',reglas[nombre],"  =  ",profundidad)
        r_nombre.append(nombre)
        r_comprimida.append(reglas[nombre])
        r_incidencias.append(int(incidencia[nombre]))  # lista de incidencias
        r_longitud.append(profundidad)    # Inicialmente cargamos con "0"
        r_extendida.append([])               # Inicialmente cargamos con listas vacias

    # Descomprimimos reglas
    resultado = descomprimir_sequitur(lista_original,r_nombre,r_comprimida,r_longitud,r_extendida)

    if len(lista_original)!=len(resultado[1][0]):
        return "Error"

    r_longitud = resultado[0]
    r_extendida = resultado[1]
    
    # R0 siempre tiene nimero de incidencias = 1
    r_incidencias[0] = 1

    return r_nombre, r_comprimida, r_incidencias, r_longitud, r_extendida
    #return r_incidencias,incidencia
    

#### Ejemplo

In [ ]:
ejemplo = ["a","a","b","c","a","a","c","b","c"]
generador_de_reglas(ejemplo)

In [ ]:
ejemplo = ["a","a","b","c","a","a","c","b","c"]
generador_de_reglas(ejemplo)

# Validacion de Reglas de Cada Cliente

Aplicamos el algoritmo de sequitur para inferir las reglas gramaticales dado por los codigos de los comportamientos, que nos generara "reglas", definida por los simbolos que se repiten en la secuencia.


### Funciones de Apoyo

#### Generador de sequencias aleatorias

In [ ]:
from random import choice
import time


def random_sequence(seq_original):
    # Fixing random state for reproducibility
    semilla = int(time.time()*10000000)
    np.random.seed(int(str(semilla)[-9:]))
    
    tags = list(np.unique(seq_original))        # Etiquetas de unicas
    long = len(seq_original)                    # Numero de perfiles unicos del cliente
    elementos = {tag:0 for tag in tags}         # Generamos lista aleatoria
    for i in range(len(seq_original)):    
        simbolo = ejemplo[i]
        elementos[simbolo] += 1

    sequences = []

    for i in range(long):
        simbolo_aleatorio = choice(tags)        # Generamos simbolo aleatorio
        elementos[simbolo_aleatorio] -= 1
        if elementos[simbolo_aleatorio] == 0:
            tags.remove(simbolo_aleatorio)
        sequences.append(simbolo_aleatorio)

    return sequences

In [ ]:
random_sequence(ejemplo)

#### Informacion de la Regla Original

    #-------------------------------------------#
    # POSICION      Reglas_originales[POSICION] #
    #-------------------------------------------#
    #    0              Nombres                 #
    #    1              Comprimidas             #
    #    2              Incidencias             #
    #    3              Longitud                #
    #    4              Extendido               #
    #-------------------------------------------#

In [ ]:
# Datos de la lista Original

reglas_originales = generador_de_reglas(ejemplo)

reglas_originales_nombre = reglas_originales[0]
reglas_originales_comprimidas = reglas_originales[1]
reglas_originales_incidencias = reglas_originales[2]
reglas_originales_longitud = reglas_originales[3]
reglas_originales_extendido = reglas_originales[4]

#### Generador de Muestra para validar Reglas

In [ ]:
# lista original,numeros
def crear_muestra_reglas(lista_original,num = 100):
    
    # Datos de la lista Original
    reglas_originales = generador_de_reglas(lista_original)
    reglas_originales_nombre = reglas_originales[0]
    reglas_originales_comprimidas = reglas_originales[1]
    reglas_originales_incidencias = reglas_originales[2]
    reglas_originales_longitud = reglas_originales[3]
    reglas_originales_extendido = reglas_originales[4]
    
    # Creamos dataframe (nombre_regla, numero_incidencias_en_cada_iteracion, numero_de_iteracion)
    matriz = [[], [], []]


    for i in range(num):                              # para *num* repeticiones
        
        sequencia_aleatoria = random_sequence(lista_original)                # Creamos la lista Aleatoria
        
        # Datos de la lista Aleatoria
        reglas_aleatorias = generador_de_reglas(sequencia_aleatoria)
        reglas_aleatorias_extendidas = reglas_aleatorias[4]  
        reglas_aleatorias_incidencias = reglas_aleatorias[2] 
        
        # Para cada regla de las regla_originales
        for j in range(len(reglas_originales_nombre)):
            
            # Para cada REGLA de las "reglas_originales"
            
            # Buscamos si "REGLA original" apareces entre las "reglas_aleatorias_extendidas" y extraemos su numero de incidencias
            numero_de_incidencia = 0
            for k in range(len(reglas_aleatorias_extendidas)):
                # Si la REGLA (reglas_aleatorias_extendidas) se encuentra en lista de "reglas_aleatorias_extendidas"
                if reglas_originales_extendido[j] == reglas_aleatorias_extendidas[k]:
                    #print(True)
                    numero_de_incidencia = int(reglas_aleatorias_incidencias[k])
        
            # Cargamos fila de la lista Matriz
            matriz[0].append(reglas_originales_nombre[j])   # Guardamos nombre en la lista                
            matriz[1].append(numero_de_incidencia)          # Guardamos numero de iteracion
            matriz[2].append(i)                             # Guardamos numero de iteracion
    
    return matriz
            
            

## Grafico para cada Cliente

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math


    Para cada cliente
        
        Obtener lista de Reglas y DAtos
        Obtener muestra a validar (Muestra)
        
        Para cada Regla de la Muestra
            imprimir tabla agrupada por numero de incidencias
            plotear

In [ ]:
# Clientes - Coordenada_X - Coordenada_Y  - Longitud - Regla_extendida
resultados_list = [[],[],[],[],[]]
resultados_list2=open("Simulaciones_Seq",'w')         # File de Entropias
resultados_list2.write('Cliente,Coordenada_X,Coordenada_Y,Distribucion,Longitud,Regla_extendida\n')    


entropia=open("Entropia.csv",'w')         # File de Entropias
entropia.write('Cliente,Entropias\n')     

for i in range(len(clientes)):
    
    ejemplo = corpus[i]
    # Datos de la regla
    reglas_originales = generador_de_reglas(ejemplo)
    reglas_originales_nombre = reglas_originales[0]        # para la entropia
    reglas_originales_comprimidas = reglas_originales[1]  
    reglas_originales_incidencias = reglas_originales[2]   # para la entropia
    reglas_originales_longitud = reglas_originales[3]
    reglas_originales_extendido = reglas_originales[4]
    
    #print("Para la entropia: ",reglas_originales_nombre,reglas_originales_incidencias)

    # CALCULAMOS LA ENTROPIA DE LAS REGLAS DE CADA CLIENTE
    base = 2
    n_labels = len(reglas_originales_incidencias)
    probabilidades = [] # probabilidades = array(reglas_originales_incidencias) / n_labels
    for num in reglas_originales_incidencias:
        probabilidades.append(num/n_labels)

    n_classes = np.count_nonzero(probabilidades)
    
    ent = 0.
    
    if n_classes <= 1:
        ent = 0.
    else:
        ent = 0.
        # funcion de la entropia
        for k in probabilidades:
            ent -= k * log(k, base)

    entropia.write(clientes[i]+","+str(ent)+"\n") #
    
    
    # Preparamos resumen de data para plotear (resultados_list)
    # PREPARAMOS DATOS PARA PLOTEAR
    
    # Creamos dataframe (nombre_regla, numero_incidencias_en_cada_iteracion, numero_de_iteracion)
    m = crear_muestra_reglas(ejemplo,1000)
    muestra = pd.DataFrame(data= {'Nombre': m[0], 'Incidencias': m[1], 'Iteracion': m[2]})
    #print(muestra.head(10))
    
    # Para cada Regla
    for j in range(len(reglas_originales_nombre)):
        #print("Regla: ",j,"       ==> ",reglas_originales_extendido)
        #print("Regla: ",reglas_originales_nombre[j])
        
        # Muestra de solo un aregla y distribucion de una regla
        submuestra = muestra.loc[muestra['Nombre'] == reglas_originales_nombre[j]]
        distribucion = submuestra.groupby(['Incidencias']).count()      # Us por incidencias
        #print(ditribucion)
        
        #xx = distribucion["Nombre"].tolist()
        yy = distribucion["Iteracion"].tolist()
        xx = [str(i) for i in list(distribucion.index)]

        resultados_list[0].append(clientes[i])
        resultados_list[1].append(xx)
        resultados_list[2].append(yy)
        resultados_list[3].append(len(reglas_originales_extendido[j]))
        resultados_list[4].append(reglas_originales_extendido[j])
        
        string = str(clientes[i]) + "," + str(xx) + "," + str(yy) + "," + str(len(reglas_originales_extendido[j])) + "," + str(reglas_originales_extendido[j])
        resultados_list2.write(string+"\n") #
        
    if i % 30000 == 0:
        print("Cliente: ",i)
        #if i == 200:
         #   break;

entropia.flush()
entropia.close()

resultados_list2.flush()
resultados_list2.close()

## Entropia de Reglas de cada Cliente

In [ ]:
Entropies = pd.read_csv("Entropia.csv", sep=",", header=0)
Entropies.head()

In [ ]:
commutes = pd.Series(Entropies["Entropias"])
#commutes.plot.hist(grid=True, bins=90, rwidth=0.5,color='#607c8e')

commutes.plot.hist(grid=True, color='#607c8e')

#plt.vlines(entropia)
plt.title('Comportamiento de la entropia')
plt.xlabel('Entropy')
plt.ylabel('Num. Clientes')
plt.grid(axis='y', alpha=0.85)

In [ ]:

fig = plt.figure(figsize=(5,2))
ax = fig.add_subplot(111, frameon=True)

ax.hist(entropia)

In [ ]:
min(entropia),max(entropia),len(entropia)

In [ ]:
#ages = pd.Series([1, 1, 3, 5, 8, 10, 12, 15, 18, 18, 19, 20, 25, 30, 40, 51, 52])
#bins = (0, 10, 13, 18, 21, np.inf)  # The edges
#labels = ('child', 'preteen', 'teen', 'military_age', 'adult')
#groups = pd.cut(ages, bins=bins, labels=labels)

In [ ]:
# Ploteamos

In [ ]:
longitud_de_regla = 2

for i in range(len(resultados_list[0])):
    
    # Clientes - Coordenada_X - Coordenada_Y - Distribucion - Longitud - Regla_extendida
    cliente = resultados_list[0][i]
    xx = resultados_list[1][i]
    yy = resultados_list[2][i]
    longitud = resultados_list[3][i]
    regla = resultados_list[4][i]
    
    print("Cliente: ",i)
    
    if longitud_de_regla == longitud:
        print("Cliente: ",cliente)
        print("Word", regla)
        
        fig = plt.figure(figsize=(5,2))
        ax = fig.add_subplot(111, frameon=True)

        ax.bar(xx,yy)
        ax.set_xlabel("Word Occurence")
        ax.set_ylabel('Frecuencia')
        plt.show()

In [ ]:
import pandas as pd

# Generate data on commute times.
size, scale = 1000, 10
commutes = pd.Series(np.random.gamma(scale, size=size) ** 1.5)

commutes.plot.hist(grid=True, bins=20, rwidth=0.9,
                   color='#607c8e')
plt.title('Commute Times for 1,000 Commuters')
plt.xlabel('Counts')
plt.ylabel('Commute Time')
plt.grid(axis='y', alpha=0.75)

In [ ]:
commutes = pd.Series(entropia2)
commutes.plot.hist(grid=True, bins=10, rwidth=0.9,
                   color='#607c8e')
plt.title('Commute Times for 1,000 Commuters')
plt.xlabel('Counts')
plt.ylabel('Commute Time')
plt.grid(axis='y', alpha=0.75)

In [ ]:
# Reglas validas
# Cliente  Regla ocurrencias
reglas_validas = [[], [], []]

for i in range(len(clientes)):
    # Extraer Reglas validad  == funcion get rules
    # para cada regla
    #    
    
        reglas_validas[0].append(clientes[i])
        reglas_validas[1].append(i)
        reglas_validas[2].append(i*i)

In [ ]:
reglas_validas

pd.DataFrame(data= {'Cliente':reglas_validas[0], 'Regla': reglas_validas[1], 'Incidencias': reglas_validas[2]})